In [2]:
# import các hàm cần thiết

import spectral.io.envi as envi		# hàm để đọc file .hdr và file .img
import numpy as np 					# để thao tác với ma trận
from spectral import open_image, imshow 	# để hiển thị hình ảnh
import pandas as pd                         # để đọc file excel
try:
    from typing import LiteralString
except ImportError:
    from typing_extensions import LiteralString
from sklearn.metrics import mean_squared_error #để tính Mean squared error
from sklearn import tree
import os
from sklearn.ensemble import RandomForestRegressor
from typing import Literal
import torch.nn as nn
import torch
import pickle
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


In [3]:
# mở file .hdr và file .img
img = envi.open("./data/spectral_image/hyper_20220913_3cm.hdr", "./data/spectral_image/hyper_20220913_3cm.img")

In [4]:
# xem các thông tin cơ bản của file
img

	Data Source:   '././data/spectral_image/hyper_20220913_3cm.img'
	# Rows:           6478
	# Samples:        5287
	# Bands:           122
	Interleave:        BSQ
	Quantization:   8 bits
	Data format:     uint8

In [5]:
img[444, 444]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [6]:
# xem kích cỡ của ma trận
img.shape

(6478, 5287, 122)

In [7]:
def get_bands(row_num, col_num):
    return img[row_num, col_num]

In [8]:
# hiển thị hình ảnh chụp được
# view = imshow(img)

In [9]:
# hàm này trả về các pixel nằm trong hình vuông có tọa độ (row_num, col_num) là tâm
# và độ dài cạnh là (2 * scopes + 1)
def pixel_in_scope(row_num, col_num, scopes):
    pixel_in_scope = set()

    for scope in range(1, scopes + 1):
        
        for i in range(row_num - scope, row_num + scope + 1):
            pixel_in_scope.add((i, col_num + scope))
            pixel_in_scope.add((i, col_num - scope))
            
        for i in range(col_num - scope, col_num + scope + 1):
            pixel_in_scope.add((row_num + scope, i))
            pixel_in_scope.add((row_num - scope, i))
            
    pixel_in_scope.add((row_num, col_num))
    return pixel_in_scope

In [10]:
# lấy thông tin về map
with open("./data/spectral_image/hyper_20220913_3cm.hdr", "r") as file:
    list_lines = file.read().split("\n")
    list_lines = filter(lambda line: "map info" in line, list_lines)
    map_infor = list(list_lines)[0]


In [11]:
map_infor

'map info = {UTM, 1.000, 1.000, 530499.467, 2355871.685, 3.0000000000e-002, 3.0000000000e-002, 48, North, WGS-84, units=Meters}'

In [12]:
# thông tin của map, tạm thời được đưa vào bằng tay
# "x1_pixel": tọa độ pixel x của điểm 1
# "y1_pixel": tọa độ pixel y của điểm 1
# "x1_axes": tọa độ trái đất x của điểm 1
# "y1_axes": tọa độ trái đất y của điểm 1
# "x_resolution": độ phân giải x của map
# "y_resolution": độ phân giải y của map
map_infor = {
    "col_1": 1.000,
    "row_1": 1.000,
    "east_1": 530499.467,
    "north_1": 2355871.685,
    "col_resolution": 3.0000000000e-002,
    "row_resolution": 3.0000000000e-002
}

In [13]:
map_infor

{'col_1': 1.0,
 'row_1': 1.0,
 'east_1': 530499.467,
 'north_1': 2355871.685,
 'col_resolution': 0.03,
 'row_resolution': 0.03}

In [14]:
# công thức lấy tọa độ trái đất khi biết tạo độ trái đất của điểm có tạo độ pixel (1, 1)
# (East - 530499.467 ) / 0.03 = số cột
# (2355871.685 - north) / 0.03 = cố hàng
# suy ra:
# East = 530499.467 + so cột * 0.03
# North = 2355871.685 - so hàng * 0.03 

In [15]:
# từ tọa độ pixel, lấy tọa độ trái đất
def get_axes(row_num, col_num):
    east = map_infor["east_1"] + col_num * map_infor["col_resolution"]  
    north = map_infor["north_1"] - row_num * map_infor["row_resolution"]
    return (north, east)
    

In [16]:
# từ tọa độ trái đất, lấy tọa độ pixel
def get_pixel(north, east):
    col_num = (east - map_infor["east_1"]) / map_infor["col_resolution"]
    row_num = (map_infor["north_1"] - north) / map_infor["row_resolution"]
    
    col_num = int(round(col_num, 0))
    row_num = int(round(row_num, 0))
    return (row_num, col_num)

In [18]:
# đọc file excel chứa data của NPK và tọa độ Trái Đất
data_df = pd.read_excel("./data/spectral_image/DATA_Mua2_PhuTho_2022_3.xlsx")

In [19]:
data_df

,Treatment,Replicate,Sub-Replicate,East,North,Elevation,Chlorophyll,Rice_Height,Spectral_number,Digesion (g),...,Chlorophyll-a.1,Unnamed: 22,Wet weight (g),Dried weight (g),% Moiture,Digesion (g).1,P conc. (mg/kg).2,K conc. (mg/kg).2,N conc. (mg/kg).2,Chlorophyll-a.2
0,9/8/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,23/9/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,T1,1.0,530606.899,2355789.223,15.195,35.2,60.0,52,0.2497,...,35.8,T1.1,2355788.259,530607.202,15.180,0.2496,4408.453526,11414.262821,1366.862450,31.7
2,NaN,NaN,2.0,530602.233,2355788.619,15.198,38.1,58.0,53,0.2492,...,34.0,T1.2,2355787.076,530602.173,15.187,0.2497,4888.165799,13192.831398,1696.964518,33.6
3,NaN,NaN,3.0,530597.380,2355787.695,15.211,30.7,64.0,54,0.2509,...,34.9,T1.3,2355786.113,530597.674,15.209,0.2494,4457.999198,14094.827586,1328.177670,32.4
4,NaN,T2,1.0,530592.844,2355790.036,15.194,31.3,63.0,55,0.2500,...,29.8,T2.1,2355789.548,530594.048,15.217,0.2495,4356.713427,13018.036072,4760.907380,30.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,NaN,BC2,NaN,530553.290,2355759.834,15.213,35.4,70.0,107,0.2502,...,33.2,BC2,2355760.894,530555.404,15.207,0.2495,5666.833667,15003.006012,NaN,32.4
75,NaN,BC3,NaN,530592.596,2355733.669,15.178,42.1,70.0,108,0.2500,...,40.0,BC3,2355732.521,530591.790,15.185,0.2500,5459.300000,10918.000000,NaN,41.4
76,NaN,BC4,NaN,530622.914,2355777.478,15.030,43.9,68.0,109-110,0.2500,...,41.8,BC4,2355776.865,530622.044,15.024,0.2504,6248.901757,13170.926518,NaN,35.9
77,NaN,BC5,NaN,530618.803,2355796.165,15.220,41.4,75.0,111,0.2500,...,35.0,BC5,2355798.179,530620.573,15.186,0.2503,6137.135437,12318.218138,NaN,29.6


In [20]:
data_df.columns

Index(['Treatment', 'Replicate', 'Sub-Replicate', 'East', 'North', 'Elevation',
       'Chlorophyll', 'Rice_Height', 'Spectral_number', 'Digesion (g)',
       'P conc. (mg/kg)', 'K conc. (mg/kg)', 'N conc. (mg/kg)',
       'Chlorophyll-a', 'Unnamed: 14', 'East.1', 'North.1', 'Elevation.1',
       'P conc. (mg/kg).1', 'K conc. (mg/kg).1', 'N conc. (mg/kg).1',
       'Chlorophyll-a.1', 'Unnamed: 22', 'Wet weight (g)', 'Dried weight (g)',
       '% Moiture', 'Digesion (g).1', 'P conc. (mg/kg).2', 'K conc. (mg/kg).2',
       'N conc. (mg/kg).2', 'Chlorophyll-a.2'],
      dtype='object')

In [21]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Treatment          1 non-null      object 
 1   Replicate          30 non-null     object 
 2   Sub-Replicate      72 non-null     float64
 3   East               78 non-null     float64
 4   North              78 non-null     float64
 5   Elevation          78 non-null     float64
 6   Chlorophyll        77 non-null     float64
 7   Rice_Height        77 non-null     float64
 8   Spectral_number    77 non-null     object 
 9   Digesion (g)       77 non-null     float64
 10  P conc. (mg/kg)    77 non-null     float64
 11  K conc. (mg/kg)    77 non-null     float64
 12  N conc. (mg/kg)    77 non-null     float64
 13  Chlorophyll-a      77 non-null     float64
 14  Unnamed: 14        77 non-null     object 
 15  East.1             76 non-null     float64
 16  North.1            76 non-nu

In [22]:
# Lấy cột dữ liệu đo được của ngày 13/9/2022
data_df_13_09_2022 = data_df.loc[1:, "Unnamed: 14":"Chlorophyll-a.1"]

In [23]:
data_df_13_09_2022.rename(columns={
    "Unnamed: 14": "type_of_field"
}, inplace=True)

In [24]:
# Bỏ đi những dòng bị thiếu tên điểm 
data_df_13_09_2022 = data_df_13_09_2022[~pd.isna(data_df_13_09_2022["type_of_field"])]

In [25]:
data_df_13_09_2022

,type_of_field,East.1,North.1,Elevation.1,P conc. (mg/kg).1,K conc. (mg/kg).1,N conc. (mg/kg).1,Chlorophyll-a.1
1,T1.1,530605.915,2355788.377,15.193,6098.182181,12505.992809,4311.970060,35.8
2,T1.2,530601.735,2355788.448,15.178,5037.174349,12338.677355,4787.270920,34.0
3,T1.3,530597.008,2355787.118,15.206,6471.319695,14923.786602,5286.813480,34.9
4,T2.1,530593.342,2355789.827,15.215,6433.340016,17184.242181,3066.571650,29.8
5,T2.2,530591.652,2355794.783,15.236,5679.671869,13274.309724,3433.145680,36.1
...,...,...,...,...,...,...,...,...
72,J12.3,530582.155,2355781.790,15.238,7009.292566,16528.776978,7556.516150,38.6
74,BC2,530553.718,2355758.342,15.224,5885.628743,14264.471058,2265.424250,33.2
75,BC3,530591.520,2355732.444,15.206,7505.289421,16167.664671,3268.614069,40.0
76,BC4,530622.015,2355776.121,15.086,6279.883674,10340.954673,2980.889693,41.8


In [26]:
data_df_13_09_2022.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76 entries, 1 to 77
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   type_of_field      76 non-null     object 
 1   East.1             76 non-null     float64
 2   North.1            76 non-null     float64
 3   Elevation.1        76 non-null     float64
 4   P conc. (mg/kg).1  76 non-null     float64
 5   K conc. (mg/kg).1  76 non-null     float64
 6   N conc. (mg/kg).1  76 non-null     float64
 7   Chlorophyll-a.1    76 non-null     float64
dtypes: float64(7), object(1)
memory usage: 5.3+ KB


In [27]:
# Tạo thêm cột 2 tọa độ pixel
data_df_13_09_2022["row_num"] = [None] * len(data_df_13_09_2022)
data_df_13_09_2022["col_num"] = [None] * len(data_df_13_09_2022)

In [28]:
data_df_13_09_2022

,type_of_field,East.1,North.1,Elevation.1,P conc. (mg/kg).1,K conc. (mg/kg).1,N conc. (mg/kg).1,Chlorophyll-a.1,row_num,col_num
1,T1.1,530605.915,2355788.377,15.193,6098.182181,12505.992809,4311.970060,35.8,None,None
2,T1.2,530601.735,2355788.448,15.178,5037.174349,12338.677355,4787.270920,34.0,None,None
3,T1.3,530597.008,2355787.118,15.206,6471.319695,14923.786602,5286.813480,34.9,None,None
4,T2.1,530593.342,2355789.827,15.215,6433.340016,17184.242181,3066.571650,29.8,None,None
5,T2.2,530591.652,2355794.783,15.236,5679.671869,13274.309724,3433.145680,36.1,None,None
...,...,...,...,...,...,...,...,...,...,...
72,J12.3,530582.155,2355781.790,15.238,7009.292566,16528.776978,7556.516150,38.6,None,None
74,BC2,530553.718,2355758.342,15.224,5885.628743,14264.471058,2265.424250,33.2,None,None
75,BC3,530591.520,2355732.444,15.206,7505.289421,16167.664671,3268.614069,40.0,None,None
76,BC4,530622.015,2355776.121,15.086,6279.883674,10340.954673,2980.889693,41.8,None,None


In [29]:
data_df_13_09_2022.columns

Index(['type_of_field', 'East.1', 'North.1', 'Elevation.1',
       'P conc. (mg/kg).1', 'K conc. (mg/kg).1', 'N conc. (mg/kg).1',
       'Chlorophyll-a.1', 'row_num', 'col_num'],
      dtype='object')

In [30]:
# Định nghĩa hàm tính tọa độ pixel của điểm thực đo ngoài thực nghiệm
def fill_row_num_and_col_num(index, row, df: pd.DataFrame):
    east = row["East.1"]
    north = row["North.1"]
    row_num, col_num = get_pixel(north, east)
    df.at[index, "row_num"] = row_num
    df.at[index, "col_num"] = col_num

In [31]:
# Lặp hàm tính tọa độ pixel trên cho từng dòng
for index, row in data_df_13_09_2022.iterrows():
    fill_row_num_and_col_num(index, row, data_df_13_09_2022)

In [32]:
data_df_13_09_2022

,type_of_field,East.1,North.1,Elevation.1,P conc. (mg/kg).1,K conc. (mg/kg).1,N conc. (mg/kg).1,Chlorophyll-a.1,row_num,col_num
1,T1.1,530605.915,2355788.377,15.193,6098.182181,12505.992809,4311.970060,35.8,2777,3548
2,T1.2,530601.735,2355788.448,15.178,5037.174349,12338.677355,4787.270920,34.0,2775,3409
3,T1.3,530597.008,2355787.118,15.206,6471.319695,14923.786602,5286.813480,34.9,2819,3251
4,T2.1,530593.342,2355789.827,15.215,6433.340016,17184.242181,3066.571650,29.8,2729,3129
5,T2.2,530591.652,2355794.783,15.236,5679.671869,13274.309724,3433.145680,36.1,2563,3073
...,...,...,...,...,...,...,...,...,...,...
72,J12.3,530582.155,2355781.790,15.238,7009.292566,16528.776978,7556.516150,38.6,2997,2756
74,BC2,530553.718,2355758.342,15.224,5885.628743,14264.471058,2265.424250,33.2,3778,1808
75,BC3,530591.520,2355732.444,15.206,7505.289421,16167.664671,3268.614069,40.0,4641,3068
76,BC4,530622.015,2355776.121,15.086,6279.883674,10340.954673,2980.889693,41.8,3185,4085


In [33]:
data_df_13_09_2022.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76 entries, 1 to 77
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   type_of_field      76 non-null     object 
 1   East.1             76 non-null     float64
 2   North.1            76 non-null     float64
 3   Elevation.1        76 non-null     float64
 4   P conc. (mg/kg).1  76 non-null     float64
 5   K conc. (mg/kg).1  76 non-null     float64
 6   N conc. (mg/kg).1  76 non-null     float64
 7   Chlorophyll-a.1    76 non-null     float64
 8   row_num            76 non-null     object 
 9   col_num            76 non-null     object 
dtypes: float64(7), object(3)
memory usage: 8.6+ KB


In [34]:
# Tính trung bình 122 band  
def get_average_bands(row_num, col_num, scope):
    number_points = 2 * scope + 1
    band_in_scope = np.zeros((number_points ** 2, 122))
    for i, (row_num, col_num) in enumerate(pixel_in_scope(row_num, col_num, scope)):
        band_in_scope[i] = img[row_num, col_num, :].reshape(122)
    return np.average(band_in_scope, axis=0)

In [35]:
def generate_sample(df: pd.DataFrame, type_of_output: Literal["N", "P", "K"] = None, type_of_field: Literal["T", "J", "BC"] = None):
    if type_of_output == "N":
        output_column = "N conc. (mg/kg).1"
    elif type_of_output == "K":
        output_column = "K conc. (mg/kg).1"
    elif type_of_output == "P":
        output_column = "P conc. (mg/kg).1"
    
    sample = df[["type_of_field",output_column, "row_num", "col_num"]]

    #Tạo bảng gồm giá trị của N, tọa độ pixel điểm thực nghiệm và 122 giá trị trống
    for i in range(122):
        sample[f"band_{i}"] = [None] * len(sample)

    # Thêm 122 giá trị trung bình của 122 band vào tập data sample N
    for index, row in sample.iterrows():
        row_num = row["row_num"]
        col_num = row["col_num"]
        sample.loc[index, "band_0":"band_121"] = get_average_bands(row_num, col_num, 3) 
    
    if type_of_field:
        if type(type_of_field) == list:
            total_condition = sample["type_of_field"].str.startswith(type_of_field[0])
            for field in type_of_field[1:]:
                condition = sample["type_of_field"].str.startswith(field)
                total_condition |= condition
            sample = sample[total_condition]
        else:
            sample = sample[sample["type_of_field"].str.startswith(type_of_field)] 
    sample.rename(columns={
        output_column: "target"
    }, inplace=True)
    return sample
    

In [36]:
# dùng deep learning

In [37]:
def create_X_train_Y_train(df: pd.DataFrame):
    X = df.loc[:, "band_0":"band_121"].to_numpy()
    Y = df.loc[:, "target"].to_numpy()

    X = X.astype(np.float32)
    Y = Y.astype(np.float32)

    total_sample = len(X)
    max_train = int(total_sample * 1)
    # max_val = int(total_sample * 1)

    X_train = torch.tensor(X[0:max_train])
    # X_val = torch.tensor(X[max_train:max_val])
    # X_test = torch.tensor(X[max_val:])

    Y_train = torch.tensor(Y[0:max_train])
    # Y_val = torch.tensor(Y[max_train:max_val])
    # Y_test = torch.tensor(Y[max_val:])

    Y_train = Y_train.reshape((Y_train.shape[0], 1))
    # Y_val = Y_val.reshape((Y_val.shape[0], 1))
    # Y_test = Y_test.reshape((Y_test.shape[0], 1))

    return X_train, Y_train

In [38]:
class NeutralNetwork(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(122, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 1)
        )
    
    def forward(self, x):
        N_value = self.linear_relu_stack(x)
        return N_value

In [39]:
def train_model(model, loss_fn, optimizer, X_train, Y_train, X_val, Y_val, n_epochs, min_loss=0.5):
    for epoch in range(n_epochs):
        model.train()

        y_pred = model(X_train)
        loss = loss_fn(y_pred, Y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            model.eval()
            with torch.inference_mode():
                loss = np.sqrt(loss)
                # y_pred_val = model(X_val)
                # loss_val = np.sqrt(loss_fn(y_pred_val, Y_val))
                # print(f"Epoch: {epoch} | loss train: {loss} | loss val: {loss_val}")
                print(f"Epoch: {epoch} | loss train: {loss}")
                if loss < min_loss:
                    return

In [40]:
RERUN_NN_MODEL = "N"

In [41]:
def predict_using_neutral_network(X_train, Y_train, X_target, Y_target, name_file_output, super_param={"lr": 0.0001, "weight_decay": 1e-5, "n_epochs": 40000, "stop_value": 0.5}, re_run="N"):
    loss_fn = nn.MSELoss()
    if not os.path.exists(name_file_output) or re_run == "Y" or RERUN_NN_MODEL == "Y":
        model = NeutralNetwork()
        optimizer = torch.optim.Adam(model.parameters(), lr=super_param["lr"], weight_decay=super_param["weight_decay"])
        n_epochs = 40000
        train_model(model, loss_fn, optimizer, X_train, Y_train, [], [], n_epochs, 0.5)
        torch.jit.script(model).save(name_file_output)
    else:
        model = torch.jit.load(name_file_output)
    
    model.eval()
    with torch.inference_mode():
        loss_fn = nn.MSELoss()
        Y_target_pred = model(X_target)
        loss = loss_fn(Y_target, Y_target_pred)
        return np.sqrt(loss), Y_target_pred, model

In [42]:
def predict_using_random_forest(X_train, Y_train, X_target, Y_target, super_param={}):
    clf = RandomForestRegressor()
    Y_train = Y_train.reshape(Y_train.shape[0])
    clf = clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_target)
    loss = np.sqrt(mean_squared_error(Y_target, Y_pred))
    return loss, Y_pred, clf
    

In [43]:
def predict_using_decision_tree(X_train, Y_train, X_target, Y_target, super_param={}):
    clf = tree.DecisionTreeRegressor()
    Y_train = Y_train.reshape(Y_train.shape[0])
    clf = clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_target)
    loss = np.sqrt(mean_squared_error(Y_target, Y_pred))
    return loss, Y_pred, clf
    

In [44]:
#Dự đoán Nito

In [45]:
target_value = "N"
train_field = "T"
sample = generate_sample(data_df_13_09_2022, target_value, train_field)
X_train, Y_train = create_X_train_Y_train(sample)
sample_target = generate_sample(data_df_13_09_2022, target_value, "BC")
X_target, Y_target = create_X_train_Y_train(sample_target)
super_param={"lr": 0.01, "weight_decay": 1e-5, "n_epochs": 40000, "stop_value": 0.5}
re_run = "N"
loss_NN, pred_NN, model_NN_1 = predict_using_neutral_network(X_train, Y_train, X_target, Y_target, f"{'_'.join(list(train_field))}_model_predict_{target_value}.pt", super_param, re_run)
print(f"{loss_NN=}")
print(f"{pred_NN=}")
loss_RF, pred_RF, clf_RF_1 = predict_using_random_forest(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_RF=}")
print(f"{pred_RF=}")
loss_DT, pred_DT, clf_DT_1 = predict_using_decision_tree(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_DT=}")
print(f"{pred_DT=}")

loss_NN=tensor(2176.3716)
pred_NN=tensor([[4883.1201],
        [4883.1201],
        [4883.1201],
        [4883.1201]])
loss_RF=1946.4111055899618
pred_RF=array([4952.68949951, 4474.99051636, 4154.02887451, 4719.08747192])
loss_DT=3257.7570008411276
pred_DT=array([6590.18115234, 4787.27099609, 7302.34228516, 4124.00146484])


In [ ]:
target_value = "N"
train_field = "J"
sample = generate_sample(data_df_13_09_2022, target_value, train_field)
X_train, Y_train = create_X_train_Y_train(sample)
sample_target = generate_sample(data_df_13_09_2022, target_value, "BC")
X_target, Y_target = create_X_train_Y_train(sample_target)
super_param={"lr": 0.0001, "weight_decay": 1e-5, "n_epochs": 40000, "stop_value": 0.5}
re_run = "N"
loss_NN, pred_NN, model_NN_2 = predict_using_neutral_network(X_train, Y_train, X_target, Y_target, f"{'_'.join(list(train_field))}_model_predict_{target_value}.pt", super_param, re_run)
print(f"{loss_NN=}")
print(f"{pred_NN=}")
loss_RF, pred_RF, clf_RF_2 = predict_using_random_forest(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_RF=}")
print(f"{pred_RF=}")
loss_DT, pred_DT, clf_DT_2 = predict_using_decision_tree(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_DT=}")
print(f"{pred_DT=}")

loss_NN=tensor(5243.0356)
pred_NN=tensor([[ 4240.9570],
        [10539.3779],
        [ 9331.4678],
        [ 6046.7734]])
loss_RF=2753.264126076129
pred_RF=array([5265.88132568, 5073.52100342, 5654.38979492, 5764.1394165 ])
loss_DT=1874.6061625965237
pred_DT=array([3289.6484375 , 4605.5078125 , 4649.16210938, 5365.20800781])


In [47]:
target_value = "N"
train_field = ["T", "J"]
sample = generate_sample(data_df_13_09_2022, target_value, train_field)
X_train, Y_train = create_X_train_Y_train(sample)
sample_target = generate_sample(data_df_13_09_2022, target_value, "BC")
X_target, Y_target = create_X_train_Y_train(sample_target)
super_param={"lr": 0.0001, "weight_decay": 1e-5, "n_epochs": 40000, "stop_value": 0.5}
re_run = "N"
loss_NN, pred_NN, model_NN_3 = predict_using_neutral_network(X_train, Y_train, X_target, Y_target, f"{'_'.join(list(train_field))}_model_predict_{target_value}.pt", super_param, re_run)
print(f"{loss_NN=}")
print(f"{pred_NN=}")
loss_RF, pred_RF, clf_RF_3 = predict_using_random_forest(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_RF=}")
print(f"{pred_RF=}")
loss_DT, pred_DT, clf_DT_3 = predict_using_decision_tree(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_DT=}")
print(f"{pred_DT=}")

loss_NN=tensor(3182.8525)
pred_NN=tensor([[5406.8696],
        [3563.7212],
        [2186.8430],
        [7931.9058]])
loss_RF=2016.4160567761423
pred_RF=array([4874.9618042 , 4526.21264526, 4191.45638428, 4991.74222412])
loss_DT=2875.489864752265
pred_DT=array([4802.86523438, 1875.54492188, 2551.82226562, 7411.33251953])


In [48]:
# Dự đoán Photpho

In [49]:
target_value = "P"
train_field = "T"
sample = generate_sample(data_df_13_09_2022, target_value, train_field)
X_train, Y_train = create_X_train_Y_train(sample)
sample_target = generate_sample(data_df_13_09_2022, target_value, "BC")
X_target, Y_target = create_X_train_Y_train(sample_target)
super_param={"lr": 0.0001, "weight_decay": 1e-5, "n_epochs": 40000, "stop_value": 0.5}
re_run = "N"
loss_NN, pred_NN, model_NN_4 = predict_using_neutral_network(X_train, Y_train, X_target, Y_target, f"{'_'.join(list(train_field))}_model_predict_{target_value}.pt", super_param, re_run)
print(f"{loss_NN=}")
print(f"{pred_NN=}")
loss_RF, pred_RF, clf_RF_4 = predict_using_random_forest(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_RF=}")
print(f"{pred_RF=}")
loss_DT, pred_DT, clf_DT_4 = predict_using_decision_tree(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_DT=}")
print(f"{pred_DT=}")

KeyboardInterrupt: 

In [ ]:
target_value = "P"
train_field = "J"
sample = generate_sample(data_df_13_09_2022, target_value, train_field)
X_train, Y_train = create_X_train_Y_train(sample)
sample_target = generate_sample(data_df_13_09_2022, target_value, "BC")
X_target, Y_target = create_X_train_Y_train(sample_target)
super_param={"lr": 0.0001, "weight_decay": 1e-5, "n_epochs": 40000, "stop_value": 0.5}
re_run = "N"
loss_NN, pred_NN, model_NN_5 = predict_using_neutral_network(X_train, Y_train, X_target, Y_target, f"{'_'.join(list(train_field))}_model_predict_{target_value}.pt", super_param, re_run)
print(f"{loss_NN=}")
print(f"{pred_NN=}")
loss_RF, pred_RF, clf_RF_5 = predict_using_random_forest(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_RF=}")
print(f"{pred_RF=}")
loss_DT, pred_DT, clf_DT_5 = predict_using_decision_tree(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_DT=}")
print(f"{pred_DT=}")

loss_NN=tensor(662.5338)
pred_NN=tensor([[7080.9502],
        [7788.8506],
        [6257.0947],
        [7120.4175]])
loss_RF=668.2471671467788
pred_RF=array([6493.85219727, 6466.68469238, 6673.04002441, 6196.55186523])
loss_DT=687.7842560589818
pred_DT=array([6219.20019531, 6219.20019531, 6219.20019531, 6273.33447266])


In [ ]:
target_value = "P"
train_field = ["T", "J"]
sample = generate_sample(data_df_13_09_2022, target_value, train_field)
X_train, Y_train = create_X_train_Y_train(sample)
sample_target = generate_sample(data_df_13_09_2022, target_value, "BC")
X_target, Y_target = create_X_train_Y_train(sample_target)
super_param={"lr": 0.0001, "weight_decay": 1e-5, "n_epochs": 40000, "stop_value": 0.5}
re_run = "N"
loss_NN, pred_NN, model_NN_6 = predict_using_neutral_network(X_train, Y_train, X_target, Y_target, f"{'_'.join(list(train_field))}_model_predict_{target_value}.pt", super_param, re_run)
print(f"{loss_NN=}")
print(f"{pred_NN=}")
loss_RF, pred_RF, clf_RF_6 = predict_using_random_forest(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_RF=}")
print(f"{pred_RF=}")
loss_DT, pred_DT, clf_DT_6 = predict_using_decision_tree(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_DT=}")
print(f"{pred_DT=}")

loss_NN=tensor(652.0106)
pred_NN=tensor([[6093.7886],
        [6544.2456],
        [5840.0708],
        [5889.4331]])
loss_RF=832.1828571331037
pred_RF=array([6273.42408691, 5956.83768066, 6356.4172168 , 6159.40990234])
loss_DT=987.4379424914434
pred_DT=array([6336.36523438, 5679.671875  , 5679.90380859, 6688.50146484])


In [ ]:
#Dự đoán Kali

In [ ]:
target_value = "K"
train_field = "T"
sample = generate_sample(data_df_13_09_2022, target_value, train_field)
X_train, Y_train = create_X_train_Y_train(sample)
sample_target = generate_sample(data_df_13_09_2022, target_value, "BC")
X_target, Y_target = create_X_train_Y_train(sample_target)
super_param={"lr": 0.0001, "weight_decay": 1e-5, "n_epochs": 40000, "stop_value": 0.5}
re_run = "N"
loss_NN, pred_NN, model_NN_7 = predict_using_neutral_network(X_train, Y_train, X_target, Y_target, f"{'_'.join(list(train_field))}_model_predict_{target_value}.pt", super_param, re_run)
print(f"{loss_NN=}")
print(f"{pred_NN=}")
loss_RF, pred_RF, clf_RF_7 = predict_using_random_forest(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_RF=}")
print(f"{pred_RF=}")
loss_DT, pred_DT, clf_DT_7 = predict_using_decision_tree(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_DT=}")
print(f"{pred_DT=}")

loss_NN=tensor(5711.0786)
pred_NN=tensor([[10922.0342],
        [13396.3223],
        [16432.9727],
        [18306.7734]])
loss_RF=3694.8214864715424
pred_RF=array([13464.63213867, 13772.40828125, 13590.90125   , 15812.79043945])
loss_DT=3338.350623621998
pred_DT=array([12338.67773438, 12474.95019531, 11111.66796875, 14836.73925781])


In [ ]:
target_value = "K"
train_field = "J"
sample = generate_sample(data_df_13_09_2022, target_value, train_field)
X_train, Y_train = create_X_train_Y_train(sample)
sample_target = generate_sample(data_df_13_09_2022, target_value, "BC")
X_target, Y_target = create_X_train_Y_train(sample_target)
super_param={"lr": 0.0001, "weight_decay": 1e-5, "n_epochs": 40000, "stop_value": 0.5}
re_run = "N"
loss_NN, pred_NN, model_NN_8 = predict_using_neutral_network(X_train, Y_train, X_target, Y_target, f"{'_'.join(list(train_field))}_model_predict_{target_value}.pt", super_param, re_run)
print(f"{loss_NN=}")
print(f"{pred_NN=}")
loss_RF, pred_RF, clf_RF_8 = predict_using_random_forest(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_RF=}")
print(f"{pred_RF=}")
loss_DT, pred_DT, clf_DT_8 = predict_using_decision_tree(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_DT=}")
print(f"{pred_DT=}")

loss_NN=tensor(4367.7793)
pred_NN=tensor([[17322.0488],
        [17014.1094],
        [12872.3936],
        [17410.7988]])
loss_RF=4379.811082007371
pred_RF=array([15213.80414062, 15905.12920898, 16115.84916992, 16187.79568359])
loss_DT=4598.859147341731
pred_DT=array([17658.5078125 , 18079.92773438, 13696.04296875, 17302.        ])


In [ ]:
target_value = "K"
train_field = ["T", "J"]
sample = generate_sample(data_df_13_09_2022, target_value, train_field)
X_train, Y_train = create_X_train_Y_train(sample)
sample_target = generate_sample(data_df_13_09_2022, target_value, "BC")
X_target, Y_target = create_X_train_Y_train(sample_target)
super_param={"lr": 0.0001, "weight_decay": 1e-5, "n_epochs": 40000, "stop_value": 0.5}
re_run = "N"
loss_NN, pred_NN, model_NN_9 = predict_using_neutral_network(X_train, Y_train, X_target, Y_target, f"{'_'.join(list(train_field))}_model_predict_{target_value}.pt", super_param, re_run)
print(f"{loss_NN=}")
print(f"{pred_NN=}")
loss_RF, pred_RF, clf_RF_9 = predict_using_random_forest(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_RF=}")
print(f"{pred_RF=}")
loss_DT, pred_DT, clf_DT_9 = predict_using_decision_tree(X_train, Y_train, X_target, Y_target, super_param)
print(f"{loss_DT=}")
print(f"{pred_DT=}")

loss_NN=tensor(4805.5151)
pred_NN=tensor([[10230.3145],
        [14042.5137],
        [14256.1875],
        [17175.5352]])
loss_RF=3665.188536792645
pred_RF=array([14292.93436523, 15034.72887695, 15081.22987305, 15150.77813477])
loss_DT=3511.2820268482556
pred_DT=array([12338.67773438, 12338.67773438, 12416.30175781, 14836.73925781])


In [ ]:
def save_img_to_folder(folder_path, img):
    start_ix = 0
    limit = 1000
    while start_ix < img.shape[0]:
        end_ix = start_ix + limit
        np.save(f"{folder_path}/{start_ix}-{end_ix}", img[start_ix:end_ix, :, :])
        start_ix = end_ix

In [ ]:
def load_img_from_folder(folder_path):
    files = os.listdir(folder_path)
    load_img = np.zeros((6478, 5287, 122), dtype=np.float16)
    for file_name in files:
        file_path = f"{folder_path}/{file_name}"
        start_ix, end_ix = os.path.splitext(file_name)[0].split("-")
        start_ix = int(start_ix)
        end_ix = int(end_ix)
        load_img[start_ix: end_ix] = np.load(file_path)
    return load_img
    

In [ ]:
def save_sklearn_model_to_file(file_path, model):
    with open(file_path, "wb") as file:
        pickle.dump(model, file)

In [ ]:
def predict_all(model, img):
    num_row = img.shape[0]
    num_col = img.shape[1]
    result_matrix = np.zeros((num_row, num_col))
    for row in range(num_row):
        for col in range(num_col):
            bands = img[row, col]
            format_bands = bands.astype(np.float32).reshape((1, bands.shape[0]))
            result_matrix[row, col] = model.predict(format_bands)
    return result_matrix
    

In [ ]:
def draw(result):
    # Assuming 'assigned_values' is your array with assigned values for each pixel
    # Replace this with your actual data
    assigned_values = np.random.randint(0, 255, size=(100, 100))

    # Define a colormap (you can choose any colormap from matplotlib)
    cmap = 'viridis'

    # Create a figure and axis
    fig, ax = plt.subplots()

    # Display the image with the assigned colors
    im = ax.imshow(result, cmap=cmap)

    # Add a colorbar to the right of the plot
    cbar = fig.colorbar(im, ax=ax)

    # Show the plot
    plt.show()


In [ ]:
# save_img_to_folder("./img_save", img)

In [ ]:
save_sklearn_model_to_file("./RF_save/clf_RF_1.pkl", clf_RF_1)